In [1]:
import strax
import numpy as np
import dask

In [2]:
import strax.ecosystem as es

In [3]:
es.PROVIDES

{'peak_details': <strax.ecosystem.PeakDetails at 0x4def278>,
 'peak_widths': <strax.ecosystem.PeakWidths at 0x3a12a20>,
 'raw_records': <strax.ecosystem.RawRecords at 0x7c90dd8>}

In [4]:
tg = es.PROVIDES['peak_widths'].task_graph('some_run')

In [5]:
tg

{('PeakDetails',
  '0.0.1'): (<bound method StraxExtension._compute of <strax.ecosystem.PeakDetails object at 0x0000000004DEF278>>, 'some_run', ('RawRecords',
   '0.0.1')),
 ('PeakWidths',
  '0.1'): (<bound method StraxExtension._compute of <strax.ecosystem.PeakWidths object at 0x0000000003A12A20>>, 'some_run', ('PeakDetails',
   '0.0.1')),
 ('RawRecords',
  '0.0.1'): (<bound method StraxExtension._compute of <strax.ecosystem.RawRecords object at 0x0000000007C90DD8>>, 'some_run')}

In [50]:
def find_cached_file(key, lin):
    if key == ('PeakDetails', '0.0.1'):
        return 'some_records_file.zstd'
    
def load_from_cache(filename):
    return

In [51]:
##
# Produce a new task graph with computes replaced by load from cache
# at as high a level as possible
##
self_key = ('PeakWidths', '0.1')

# Map of direct dependencies for each task
direct_deps_of = dask.optimize.cull(tg, self_key)[1]

tg_new = {}
stack = [self_key]
while len(stack):
    key = stack.pop()
    if key in tg_new:
        # We're already on the new task graph
        continue
        
    # Get full lineage of key
    # Do we have this on ice?
    lin = lineage(tg, key)
    filename = find_cached_file(key, lin)
    if filename:
        # YES, add a *load_from_cache* to the new task graph
        tg_new[key] = (load_from_cache, filename)
        pass
    else:    
        # NO, the original *compute* goes on the task graph
        # Put its dependencies on the stack.
        tg_new[key] = tg[key]
        stack.extend(direct_deps_of[key])


In [52]:
tg_new

{('PeakDetails', '0.0.1'): (<function __main__.load_from_cache>,
  'some_records_file.zstd'),
 ('PeakWidths',
  '0.1'): (<bound method StraxExtension._compute of <strax.ecosystem.PeakWidths object at 0x0000000003A12A20>>, 'some_run', ('PeakDetails',
   '0.0.1'))}

In [39]:
# Compute full lineage set for each task
def lineage(tg, k):
    ks = set(dask.optimize.cull(tg, k)[0].keys())
    ks.remove(k)
    return ks

lineage(tg, ('PeakWidths', '0.1'))

{('PeakDetails', '0.0.1'), ('RawRecords', '0.0.1')}

In [16]:
# TODO: PROVIDING algorithm must be in key!
new_tg = dict()
for k in list(reversed(dask.core.toposort(tg))):
    dep_keys = set(dask.optimize.cull(tg, k)[0].keys())
    dep_keys.remove(k)
    print(k, dep_keys)
    # Look up in db
    # If exists, put load from cache in new task graph
    # If not exists, keep compute

('PeakWidths', '0.1') {('PeakDetails', '0.0.1'), ('RawRecords', '0.0.1')}
('PeakDetails', '0.0.1') {('RawRecords', '0.0.1')}
('RawRecords', '0.0.1') set()


In [12]:
kset.remove?

In [ ]:


Hash these, look them up.
For all available datasets

In [11]:
es.PROVIDES['peak_widths'].compute

<bound method PeakWidths.compute of <strax.ecosystem.PeakWidths object at 0x0000000007A04D68>>

In [10]:
from dask.dot import dot_graph

dict(tg.dask)

{'_compute-2a4dc50f-602e-4396-b115-4a06ff7a7418': (<bound method StraxExtension._compute of <strax.ecosystem.ProcessedData object at 0x0000000007A10080>>,
  'some_run',
  '_compute-866adb99-1224-48df-99c1-6c87cfa2e5a2'),
 '_compute-866adb99-1224-48df-99c1-6c87cfa2e5a2': (<bound method StraxExtension._compute of <strax.ecosystem.RawRecords object at 0x0000000007C9CC88>>,
  'some_run'),
 '_compute-fe45645c-25c4-47ee-bd88-ae02d2e07b84': (<bound method StraxExtension._compute of <strax.ecosystem.PeakWidths object at 0x0000000007A04D68>>,
  'some_run',
  '_compute-2a4dc50f-602e-4396-b115-4a06ff7a7418')}